In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("retail.csv")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,"6,95",13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,"2,1",13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,"1,25",13085.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1048575 non-null  object 
 1   StockCode    1048575 non-null  object 
 2   Description  1044203 non-null  object 
 3   Quantity     1048575 non-null  int64  
 4   InvoiceDate  1048575 non-null  object 
 5   Price        1048575 non-null  object 
 6   Customer ID  811893 non-null   float64
 7   Country      1048575 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 64.0+ MB


In [4]:
#Checking for missing values in data
df.isnull().sum()

Invoice             0
StockCode           0
Description      4372
Quantity            0
InvoiceDate         0
Price               0
Customer ID    236682
Country             0
dtype: int64

In [5]:
#Removing the rows with missing values
df.dropna(subset= "Customer ID", inplace = True)

In [6]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [7]:
#Converting the data type to datatime format 
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 811893 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      811893 non-null  object        
 1   StockCode    811893 non-null  object        
 2   Description  811893 non-null  object        
 3   Quantity     811893 non-null  int64         
 4   InvoiceDate  811893 non-null  datetime64[ns]
 5   Price        811893 non-null  object        
 6   Customer ID  811893 non-null  float64       
 7   Country      811893 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 55.7+ MB


#### Task 1:
We will need to compute the following for each customer:  

1. The number of times a customer has made transactions. If in a single day a customer has made 3 transactions, count them as 3 separate transactions. This is the frequency in RFM.  

2. The total and average revenue per customer. To arrive at revenue, you will need to multiply the Quantity and Price columns. You will also need to clean the Price column for any data quality issues. This will become the monetary term in RFM.  

3. Lastly we will need to find the recency of the last purchase. This can be computed by finding the number of days that have elapsed from the last purchase each customer has made. You can use a base date of 01/01/2012 to compute recency. Find out the number of days elapsed from 01/01/2012 for each customer's most recent purchase.  

In [9]:
#Fixing the incorrect decimal point format for prices 
df["Price"] = df["Price"].str.replace("," , ".")

In [10]:
#Converting the data type to numeric for ease of mathematical operations
df["Price"] = pd.to_numeric(df["Price"])

In [11]:
df["Revenue"] = df["Quantity"] * df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-01-12 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-01-12 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-01-12 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-01-12 07:45:00,1.25,13085.0,United Kingdom,30.0


In [12]:
df_RFM = df.groupby(["Customer ID"], as_index = False).aggregate({"Price": "count", "Revenue": ["sum","mean"]})
df_RFM

Customer ID Price  Revenue           
                 count      sum       mean
0        12346.0    48   -64.68  -1.347500
1        12347.0   242  5408.50  22.349174
2        12348.0    51  2019.40  39.596078
3        12349.0   180  4404.54  24.469667
4        12350.0    17   334.40  19.670588
...          ...   ...      ...        ...
5919     18283.0   936  2528.65   2.701549
5920     18284.0    29   436.68  15.057931
5921     18285.0    12   427.00  35.583333
5922     18286.0    70  1188.43  16.977571
5923     18287.0   156  4177.89  26.781346

[5924 rows x 4 columns]

In [13]:
df_RFM["Total_rev"] = df_RFM["Revenue"]["sum"]
df_RFM["Avg_rev"] = df_RFM["Revenue"]["mean"]
df_RFM["Freq"] = df_RFM["Price"]["count"]

In [14]:
df_RFM.head()

Customer ID Price  Revenue            Total_rev    Avg_rev Freq
              count      sum       mean                          
0     12346.0    48   -64.68  -1.347500    -64.68  -1.347500   48
1     12347.0   242  5408.50  22.349174   5408.50  22.349174  242
2     12348.0    51  2019.40  39.596078   2019.40  39.596078   51
3     12349.0   180  4404.54  24.469667   4404.54  24.469667  180
4     12350.0    17   334.40  19.670588    334.40  19.670588   17

In [15]:
del df_RFM["Price"]
del df_RFM["Revenue"]

In [16]:
df_RFM.head()

,Customer ID,Total_rev,Avg_rev,Freq
,,,,
0,12346.0,-64.68,-1.347500,48
1,12347.0,5408.50,22.349174,242
2,12348.0,2019.40,39.596078,51
3,12349.0,4404.54,24.469667,180
4,12350.0,334.40,19.670588,17


In [17]:
df_recency = df.groupby(by = "Customer ID", as_index = False)["InvoiceDate"].max()
df_recency.columns = ["Customer ID", "LastPurchaseDate"]

df_recency["Recency"] = df_recency["LastPurchaseDate"].apply(lambda x: pd.to_datetime("2012-01-01") - x)
df_recency

,Customer ID,LastPurchaseDate,Recency
0,12346.0,2011-01-18 10:17:00,347 days 13:43:00
1,12347.0,2011-10-31 12:25:00,61 days 11:35:00
2,12348.0,2011-09-25 13:13:00,97 days 10:47:00
3,12349.0,2011-11-21 09:51:00,40 days 14:09:00
4,12350.0,2011-02-02 16:01:00,332 days 07:59:00
...,...,...,...
5919,18283.0,2011-11-30 12:59:00,31 days 11:01:00
5920,18284.0,2010-06-10 12:31:00,569 days 11:29:00
5921,18285.0,2010-02-17 10:24:00,682 days 13:36:00
5922,18286.0,2010-08-20 11:57:00,498 days 12:03:00


In [18]:
#Adding the recency column in original RFM table
df_RFM["Recency"] = df_recency["Recency"]
df_RFM

,Customer ID,Total_rev,Avg_rev,Freq,Recency
,,,,,
0,12346.0,-64.68,-1.347500,48,347 days 13:43:00
1,12347.0,5408.50,22.349174,242,61 days 11:35:00
2,12348.0,2019.40,39.596078,51,97 days 10:47:00
3,12349.0,4404.54,24.469667,180,40 days 14:09:00
4,12350.0,334.40,19.670588,17,332 days 07:59:00
...,...,...,...,...,...
5919,18283.0,2528.65,2.701549,936,31 days 11:01:00
5920,18284.0,436.68,15.057931,29,569 days 11:29:00
5921,18285.0,427.00,35.583333,12,682 days 13:36:00


In [19]:
df_RFM.shape

(5924, 5)

#### Task 2:
Once we have the RFM table, you will need to analyze this table further.  

One of the things we can analyze is to find the vintage of customers. Find out for which ranges of recency, is the company realizing around 75% of the total revenue.  

In [20]:
#Creating Segmentation of customer data by finding Deciles for Recency and Frequency.
df_RFM["Recency_Deciles"] = pd.qcut(df_RFM["Recency"], q=10)
df_RFM["Freq_Deciles"] = pd.qcut(df_RFM["Freq"], q=10)
df_RFM

,Customer ID,Total_rev,Avg_rev,Freq,Recency,Recency_Deciles,Freq_Deciles
,,,,,,,
0,12346.0,-64.68,-1.347500,48,347 days 13:43:00,"(345 days 09:39:30, 436 days 12:19:24]","(37.0, 53.0]"
1,12347.0,5408.50,22.349174,242,61 days 11:35:00,"(61 days 10:34:24.000000001, 87 days 18:32:36....","(180.0, 319.7]"
2,12348.0,2019.40,39.596078,51,97 days 10:47:00,"(87 days 18:32:36.000000017, 135 days 08:26:00]","(37.0, 53.0]"
3,12349.0,4404.54,24.469667,180,40 days 14:09:00,"(32 days 12:04:36, 43 days 13:59:12]","(114.0, 180.0]"
4,12350.0,334.40,19.670588,17,332 days 07:59:00,"(216 days 09:30:36.000000004, 345 days 09:39:30]","(16.0, 25.0]"
...,...,...,...,...,...,...,...
5919,18283.0,2528.65,2.701549,936,31 days 11:01:00,"(21 days 06:40:59.999999999, 32 days 12:04:36]","(319.7, 12780.0]"
5920,18284.0,436.68,15.057931,29,569 days 11:29:00,"(557 days 12:04:00, 1083 days 14:05:00]","(25.0, 37.0]"
5921,18285.0,427.00,35.583333,12,682 days 13:36:00,"(557 days 12:04:00, 1083 days 14:05:00]","(8.0, 16.0]"


In [21]:
df_vintage = df_RFM.groupby(by = "Recency_Deciles", as_index = False)["Total_rev"].sum()
df_vintage

,Recency_Deciles,Total_rev
,,
0,"(21 days 06:40:59.999999999, 32 days 12:04:36]",5711857.734
1,"(32 days 12:04:36, 43 days 13:59:12]",2968252.921
2,"(43 days 13:59:12, 61 days 10:34:24.000000001]",2275605.871
3,"(61 days 10:34:24.000000001, 87 days 18:32:36....",1533952.931
4,"(87 days 18:32:36.000000017, 135 days 08:26:00]",1262303.344
5,"(135 days 08:26:00, 216 days 09:30:36.000000004]",965468.294
6,"(216 days 09:30:36.000000004, 345 days 09:39:30]",586407.400
7,"(345 days 09:39:30, 436 days 12:19:24]",536943.362
8,"(436 days 12:19:24, 557 days 12:04:00]",398391.840


In [22]:
#Finding out Cumulative Sum and Total Sum of revenue and Percentage of Cumulative Sum for analysis.
df_vintage["CumSum_Rev"] = df_vintage["Total_rev"].cumsum()
df_vintage["Total_Rev_Across_Deciles"] = df_vintage["Total_rev"].sum()
df_vintage["Perc_tot_rev"] = df_vintage["CumSum_Rev"]/df_vintage["Total_Rev_Across_Deciles"]

In [23]:
#Final table for vintage of customers to identify the customers by their recency and how much revenue they bring.
df_vintage

,Recency_Deciles,Total_rev,CumSum_Rev,Total_Rev_Across_Deciles,Perc_tot_rev
,,,,,
0,"(21 days 06:40:59.999999999, 32 days 12:04:36]",5711857.734,5.711858e+06,1.640410e+07,0.348197
1,"(32 days 12:04:36, 43 days 13:59:12]",2968252.921,8.680111e+06,1.640410e+07,0.529143
2,"(43 days 13:59:12, 61 days 10:34:24.000000001]",2275605.871,1.095572e+07,1.640410e+07,0.667865
3,"(61 days 10:34:24.000000001, 87 days 18:32:36....",1533952.931,1.248967e+07,1.640410e+07,0.761375
4,"(87 days 18:32:36.000000017, 135 days 08:26:00]",1262303.344,1.375197e+07,1.640410e+07,0.838325
5,"(135 days 08:26:00, 216 days 09:30:36.000000004]",965468.294,1.471744e+07,1.640410e+07,0.897181
6,"(216 days 09:30:36.000000004, 345 days 09:39:30]",586407.400,1.530385e+07,1.640410e+07,0.932928
7,"(345 days 09:39:30, 436 days 12:19:24]",536943.362,1.584079e+07,1.640410e+07,0.965661
8,"(436 days 12:19:24, 557 days 12:04:00]",398391.840,1.623918e+07,1.640410e+07,0.989947


From above we can see that the customer in recency deciles from Row Index 0-3 are bringing around 75% of total revenue.

In [24]:
recency_cust_75rev = list(df_vintage["Recency_Deciles"].astype("str")[0:4])

#Recency Ranges of the customer who bring approx 75% of the revenue
recency_cust_75rev

['(21 days 06:40:59.999999999, 32 days 12:04:36]',
 '(32 days 12:04:36, 43 days 13:59:12]',
 '(43 days 13:59:12, 61 days 10:34:24.000000001]',
 '(61 days 10:34:24.000000001, 87 days 18:32:36.000000017]']

#### Task 3:
To find out within the subset of data identified above, for what ranges of frequency is ~65% of the revenue in the subset realized.

In [25]:
#Creating a subset of data identified above for further analysis of Frequency (The customers who generate approx 75% revenue)
df_RFM_subset = df_RFM[(df_RFM["Recency"] <= "87 days 18:32:36.000000017") & (df_RFM["Recency"] >= "21 days 06:40:59.999999999")]
df_RFM_subset.head()

,Customer ID,Total_rev,Avg_rev,Freq,Recency,Recency_Deciles,Freq_Deciles
,,,,,,,
1,12347.0,5408.50,22.349174,242,61 days 11:35:00,"(61 days 10:34:24.000000001, 87 days 18:32:36....","(180.0, 319.7]"
3,12349.0,4404.54,24.469667,180,40 days 14:09:00,"(32 days 12:04:36, 43 days 13:59:12]","(114.0, 180.0]"
10,12356.0,6373.68,44.571189,143,44 days 15:20:00,"(43 days 13:59:12, 61 days 10:34:24.000000001]","(114.0, 180.0]"
12,12358.0,3203.87,45.769571,70,24 days 13:56:00,"(21 days 06:40:59.999999999, 32 days 12:04:36]","(53.0, 76.0]"
13,12359.0,8714.89,23.239707,375,30 days 11:17:00,"(21 days 06:40:59.999999999, 32 days 12:04:36]","(319.7, 12780.0]"


In [26]:
df_RFM_subset.head()

,Customer ID,Total_rev,Avg_rev,Freq,Recency,Recency_Deciles,Freq_Deciles
,,,,,,,
1,12347.0,5408.50,22.349174,242,61 days 11:35:00,"(61 days 10:34:24.000000001, 87 days 18:32:36....","(180.0, 319.7]"
3,12349.0,4404.54,24.469667,180,40 days 14:09:00,"(32 days 12:04:36, 43 days 13:59:12]","(114.0, 180.0]"
10,12356.0,6373.68,44.571189,143,44 days 15:20:00,"(43 days 13:59:12, 61 days 10:34:24.000000001]","(114.0, 180.0]"
12,12358.0,3203.87,45.769571,70,24 days 13:56:00,"(21 days 06:40:59.999999999, 32 days 12:04:36]","(53.0, 76.0]"
13,12359.0,8714.89,23.239707,375,30 days 11:17:00,"(21 days 06:40:59.999999999, 32 days 12:04:36]","(319.7, 12780.0]"


In [27]:
#Finding Aggregate Sum of revenue for each Frequency Decile range
df_freq_dec = df_RFM_subset.groupby(by = "Freq_Deciles", as_index = False)["Total_rev"].sum()
df_freq_dec

,Freq_Deciles,Total_rev
,,
0,"(0.999, 8.0]",33444.130
1,"(8.0, 16.0]",63917.290
2,"(16.0, 25.0]",83236.670
3,"(25.0, 37.0]",134151.150
4,"(37.0, 53.0]",202161.950
5,"(53.0, 76.0]",374664.510
6,"(76.0, 114.0]",546470.850
7,"(114.0, 180.0]",992185.561
8,"(180.0, 319.7]",2157491.621


In [28]:
#Finding out the Cumulative sum, Sum Total of revenue and percentage of CumSum revenue
df_freq_dec["CumSum"] = df_freq_dec["Total_rev"].cumsum()
df_freq_dec["Total"] = df_freq_dec["Total_rev"].sum()
df_freq_dec["Perc_Total"] = df_freq_dec["CumSum"]/df_freq_dec["Total"]

In [29]:
#Final table to identify the customers by their frequency of orders who generated approx 65% of the revenue for business.
df_freq_dec

,Freq_Deciles,Total_rev,CumSum,Total,Perc_Total
,,,,,
0,"(0.999, 8.0]",33444.130,3.344413e+04,1.248967e+07,0.002678
1,"(8.0, 16.0]",63917.290,9.736142e+04,1.248967e+07,0.007795
2,"(16.0, 25.0]",83236.670,1.805981e+05,1.248967e+07,0.014460
3,"(25.0, 37.0]",134151.150,3.147492e+05,1.248967e+07,0.025201
4,"(37.0, 53.0]",202161.950,5.169112e+05,1.248967e+07,0.041387
5,"(53.0, 76.0]",374664.510,8.915757e+05,1.248967e+07,0.071385
6,"(76.0, 114.0]",546470.850,1.438047e+06,1.248967e+07,0.115139
7,"(114.0, 180.0]",992185.561,2.430232e+06,1.248967e+07,0.194579
8,"(180.0, 319.7]",2157491.621,4.587724e+06,1.248967e+07,0.367321


From above we can see that the range of frequency of customers who generate ~65% of revenue fall between (319, 12780)